In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as pplt
import random
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.neighbors import NearestNeighbors  ## This guy use 1-cos_distance as metrics

In [ ]:
## Read the dataset
df  = pd.read_csv('C:\\Users\\tanghao\\Desktop\\yelp_2k.csv',header = 0)
docs = pd.DataFrame()
docs['X'] = df['text'].copy()
docs['y'] = df['stars'].copy()

In [ ]:
## Remove the stop words then set the max_df = 1000, min_df = 2
vectorizer_optimized = CountVectorizer(stop_words = SW, max_df = 1000, min_df = 2)  ## Instantiate an object, 
Vec_optimized = vectorizer_optimized.fit_transform(list(docs['X']))

In [ ]:
## Process the data
X_optimized = Vec_optimized.toarray()

In [ ]:
## Nearest Neighbor Method
Neighbors = NearestNeighbors(n_neighbors = 5, metric='cosine') 
Neighbors.fit(X_optimized)  
distance_score, index = Neighbors.kneighbors([query_vector])

In [ ]:
## My own split method to split it
def trainTestSplit(dataframe, ratio):
    random.seed(0)
    testCount = int(len(dataframe) * (1-ratio)) + 1
    testSampleIndex = random.sample(range(0, len(dataframe)), testCount)
    trainSampleIndex = list(set(range(0,len(dataframe))) - set(testSampleIndex))
    testset = dataframe.drop(trainSampleIndex)
    trainset = dataframe.drop(testSampleIndex)
    split1 = {}
    split1['train'] = trainset
    split1['test'] = testset
    return split1 

In [ ]:
## Get the new column with 1 or 0 for 5 stars and 1 stars
docs['y'] = df['stars'].apply(lambda x: 1 if x == 5 else 0)  ## 'stars' is int64
docs['X_vector'] = pd.Series(list(X_optimized))

In [ ]:
## generate the prediction
LR = LogisticRegression(solver = 'lbfgs')
LR.fit(list(vector_train),list(label_train))
prediction_test=LR.predict(list(vector_test))
prediction_train = LR.predict(list(vector_train))

In [ ]:
## The prediction rate, This is from my own method of split
rate_test = sum(prediction_test == label_test)/len(prediction_test) ## A quick look
rate_train = sum(prediction_train == label_train)/len(prediction_train)
print(rate_test)
print(rate_train)
con_matrix_test = metrics.confusion_matrix(label_test, prediction_test)
con_matrix_test

In [ ]:
hist_df_train = pd.DataFrame(LR.predict_proba(list(vector_train)),columns = ['pred_0','pred_1'])
hist_df_test = pd.DataFrame(LR.predict_proba(list(vector_test)),columns = ['pred_0','pred_1'])
hist_df_train['label'] = pd.Series(list(label_train))
hist_df_test['label'] = pd.Series(list(label_test))
## get the histogram with positive label for test
## change the threshold may improve the accuracy of test set
fig = pplt.figure(figsize=(10,10))
pplt.title('Histogram of Predicted Scores', fontsize = 28)
pplt.hist(hist_df_test[hist_df_test['label'] == 0]['pred_1'],bins = 200, color = 'green', alpha = 0.5)
pplt.hist(hist_df_test[hist_df_test['label'] == 1]['pred_1'],bins = 200, color = 'orange', alpha = 0.5)
pplt.xlabel('Prodicted Score', fontsize = 24)
pplt.xticks(fontsize = 18)
pplt.ylabel('Count of prediction in bucket', fontsize = 24)
pplt.yticks(fontsize = 18)
pplt.show()